In [1]:
import geopandas as gpd

floripa = gpd.read_file("gvw_distritos_administrativos.shp")
floripa_utm = floripa.to_crs("EPSG:31982")
floripa_utm["geometry"] = floripa_utm.buffer(50)
floripa_buffer_50m = floripa_utm.to_crs("EPSG:4326")

In [4]:
## teste v1
import rasterio
from rasterio.mask import mask
from rasterio.transform import from_bounds

for ano in range(1985, 2025):
    url = f"https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/collection_10/lulc/coverage/brazil_coverage_{ano}.tif"

    with rasterio.open(url) as src:
        floripa = floripa_buffer_50m.to_crs(src.crs)
    
        out_image, _ = mask(
            src,
            floripa.geometry,
            crop=True,
            nodata=src.nodata
        )
    
        bounds = floripa.total_bounds
        res_x, res_y = src.res
    
        out_transform = from_bounds(
            bounds[0], bounds[1], bounds[2], bounds[3],
            out_image.shape[2],
            out_image.shape[1]
        )
    
        out_meta = {
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "count": out_image.shape[0],
            "dtype": out_image.dtype,
            "crs": src.crs,
            "transform": out_transform,
            "nodata": src.nodata
        }
    
        with rasterio.open(f"florianopolis_{ano}_v1.tif", "w", **out_meta) as dst:
            dst.write(out_image)